<a href="https://colab.research.google.com/github/aekanun2020/2022-PSU-Data-Science-for-Tourism-Course1/blob/main/Quiz2-PySpark_DataScience_onColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

# **1. ธุรกิจต้องการ ML Model ไปใช้ในการทำนายว่า ลูกค้าที่กำลังเลือกดูสินค้าอยู่ จะซื้อหรือไม่ซื้อสินค้า (รวมถึงพฤติกรรมอื่นๆ ด้วย)**

# **2. Data Understanding**

In [7]:
! wget https://storage.googleapis.com/class25jan2022/share/testfunnel.csv

--2022-02-06 13:16:26--  https://storage.googleapis.com/class25jan2022/share/testfunnel.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.240, 172.253.122.128, 172.253.63.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1501012 (1.4M) [text/csv]
Saving to: ‘testfunnel.csv’

testfunnel.csv      100%[===================>]   1.43M  --.-KB/s    in 0.01s   

2022-02-06 13:16:26 (111 MB/s) - ‘testfunnel.csv’ saved [1501012/1501012]



In [8]:
! head -3 testfunnel.csv

source,isTrueDirect,sourceKeyword,medium,isVideoAd,fullVisitorId,visitId,date,newVisits,hitReferer,hitType,hitAction_type,hitNumber,hitHour,hitMin,timeMicroSec,v2ProductName,productListName,isClick,isImpression,sessionQualityDim,timeOnScreen,timeOnSite,totalTransactionRevenue
google,true,(not provided),organic,,4988612949713423910,1489617360,20170315,,,PAGE,0,28,15,46,608771,Google Accent Insulated Stainless Steel Bottle,Category,,true,,,1897,
google,true,(not provided),organic,,4988612949713423910,1489617360,20170315,,,PAGE,0,27,15,46,605064,Google Pocket Bluetooth Speaker,Category,,true,,,1897,


In [9]:
! wc -l testfunnel.csv

10000 testfunnel.csv


In [10]:
raw_df = spark.read.option('header','true')\
.option("inferSchema" , "true")\
.format('csv').load('./testfunnel.csv')

In [11]:
raw_df.count()

9999

In [12]:
len(raw_df.columns)

24

In [13]:
raw_df.sample(0.001).toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
source,google,google,(direct),google,google.com,google,google,google,(direct),(direct),(direct),google,google,google
isTrueDirect,True,None,True,None,None,None,None,True,None,True,True,None,None,None
sourceKeyword,(not provided),(not provided),None,(not provided),None,(not provided),(not provided),(not provided),None,None,None,(not provided),(not provided),(not provided)
medium,organic,organic,(none),organic,referral,organic,organic,organic,(none),(none),(none),organic,organic,organic
isVideoAd,None,None,None,None,None,None,None,None,None,None,None,None,None,None
fullVisitorId,4988612949713423910,4990739953666415758,4990803917337260315,4990955117732204143,4991315443667882035,4991962803971844700,4992499749003799871,4992970334133037215,4993218711598105610,4993852637585406336,4993888514053304960,4994205137593013538,4995239461409228195,4995903322841177843
visitId,1490043910,1501511503,1477067425,1490136764,1489472100,1475876346,1472235854,1474396786,1487484373,1494000490,1492820576,1499898511,1486916946,1493223582
date,20170320,20170731,20161021,20170321,20170313,20161007,20160826,20160920,20170218,20170505,20170421,20170712,20170212,20170426
newVisits,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0
hitReferer,None,https://www.google.pt/,None,None,None,None,None,None,https://www.google.co.za/,None,None,https://www.google.com/,None,https://www.google.fr/


In [14]:
raw_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
source,9999,None,None,(direct),youtube.com
sourceKeyword,4042,None,None,(not provided),youtube merch
medium,9999,None,None,(none),referral
fullVisitorId,9999,4.5067648825526333E18,1.39995873491150899E18,49898505875812980,4995924842281653133
visitId,9999,1.4850613864552455E9,9020666.664979974,1470106850,1501649190
date,9999,2.016601000310031E7,4694.944823979139,20160801,20170801
newVisits,7269,1.0,0.0,1,1
hitReferer,1333,None,None,http://images.google.co.in/imgres?imgurl=https...,https://www.youtube.com/yt/about/press/
hitType,9999,None,None,EVENT,PAGE


In [15]:
raw_df.printSchema()

root
 |-- source: string (nullable = true)
 |-- isTrueDirect: boolean (nullable = true)
 |-- sourceKeyword: string (nullable = true)
 |-- medium: string (nullable = true)
 |-- isVideoAd: boolean (nullable = true)
 |-- fullVisitorId: long (nullable = true)
 |-- visitId: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- newVisits: integer (nullable = true)
 |-- hitReferer: string (nullable = true)
 |-- hitType: string (nullable = true)
 |-- hitAction_type: integer (nullable = true)
 |-- hitNumber: integer (nullable = true)
 |-- hitHour: integer (nullable = true)
 |-- hitMin: integer (nullable = true)
 |-- timeMicroSec: integer (nullable = true)
 |-- v2ProductName: string (nullable = true)
 |-- productListName: string (nullable = true)
 |-- isClick: boolean (nullable = true)
 |-- isImpression: boolean (nullable = true)
 |-- sessionQualityDim: integer (nullable = true)
 |-- timeOnScreen: string (nullable = true)
 |-- timeOnSite: integer (nullable = true)
 |-- totalTransac